<div style="font-size: 200%; font-weight: bold; color: maroon;">Red de contactos en GraphFrames</div>
<!-- Paulo Villegas, 2021 - v. 1.1 -->

Cargamos la misma red de contactos que vimos en un [_Notebook_ anterior](04%20contact%20network.ipynb) y procesamos allí con NetworkX

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import networkx as nx

%matplotlib inline

In [ ]:
spark

# Leer datos

Los datos de contactos están almacenados en un ficheros CSV. Vamos a crear un DataFrame con ellos.

In [ ]:
from pyspark.sql.types import StructType, StructField, StringType, LongType
import pyspark.sql.functions as F

In [ ]:
schema = StructType([
   StructField("ts", LongType(), True),   # contact timestamp (in seconds, unix epoch)
   StructField("id1", LongType(), False),
   StructField("id2", LongType(), False),
   StructField("group1", StringType(), True),
   StructField("group2", StringType(), True)])

In [ ]:
df_contacts = spark.read.csv('DATA/High-School_data_2013.csv.gz', sep=' ', header=False, schema=schema)

Añadimos una columna más, con la marca de tiempo del contacto, pero en formato cadena

In [ ]:
df_contacts = df_contacts.withColumn('time', F.from_unixtime('ts'))

In [ ]:
df_contacts.printSchema()

In [ ]:
df_contacts.count()

In [ ]:
df_contacts.limit(10).toPandas()

## Crear dataframes de nodos y enlaces

### nodos

Crear DataFrame de nodos. Debe tener una columna `id` con el índice del nodo. Vamos a crearla a partir del DataFrame de contactos, extrayendo los valores de las dos personas del contacto

In [ ]:
nodes1 = df_contacts.select( F.col('id1').alias('id'), F.col('group1').alias('group') )

In [ ]:
nodes2 = df_contacts.select(['id2', 'group2']).withColumnRenamed('id2', 'id').withColumnRenamed('group2', 'group')

Ahora juntamos los dos DataFrames, y nos aseguramos de eliminar duplicados

In [ ]:
nodes = nodes1.unionByName(nodes2).distinct()

### enlaces

Crear DataFrame de enlaces. Debe tener dos columnas `src` y `dst`, más los metadatos adicionales disponibles (en este caso, la marca de tiempo del contacto, en sus dos formatos)

In [ ]:
edges = df_contacts.select(['id1', 'id2', 'time', 'ts'])

In [ ]:
edges = edges.withColumnRenamed('id1', 'src').withColumnRenamed('id2', 'dst')

Veamos ejemplos de filas

In [ ]:
nodes.limit(10).toPandas()

In [ ]:
edges.limit(10).toPandas()

## Crear el grafo

In [ ]:
import graphframes as gf

In [ ]:
cn = gf.GraphFrame(nodes, edges)

In [ ]:
cn.vertices.count(), cn.edges.count()

## Grabarlo

In [ ]:
cn.vertices.write.mode('overwrite').parquet('OUT/contact-network-vertices')
cn.edges.write.mode('overwrite').parquet('OUT/contact-network-edges')

# Análisis
## Consultas generales

Intervalo de tiempo cubierto por los contactos

In [ ]:
cn.edges.select( F.min(F.col('time')).alias('start'), F.max(F.col('time')).alias('end') ).collect()

Número de contactos de cada día

In [ ]:
cn.edges.groupby(F.date_trunc('day', 'time').alias('day')).count().sort('day').toPandas()

Alumnos en cada grupo

In [ ]:
cn.vertices.groupBy('group').count().sort('group').toPandas()


<div class="alert alert-warning" role="alert" style="margin-top: 24pt; margin-bottom: 0pt;">Ejercicio: averigüe la densidad de las relaciones entre cursos, contando los contactos que hay entre alumnos de distintos cursos</div>

## Grados de los nodos

In [ ]:
d = cn.degrees.join(nodes, 'id').sort('degree', ascending=False)

Estadísticas de los grados, por cada grupo

In [ ]:
group_stats = d.groupby('group').agg(F.mean(d.degree).alias('avg'), 
                                     F.stddev(d.degree).alias('sdev'), 
                                     F.min(d.degree).alias('min'),
                                     F.max(d.degree).alias('max'))
group_stats.sort('avg').toPandas()

Nodos con grado menor que 50

In [ ]:
d.filter(d.degree < 50).show()

## Community detection

Probando con LPA y con componentes fuertemente conectados, ninguno detecta grupos interesantes

In [ ]:
%%time

result = cn.labelPropagation(maxIter=4)

In [ ]:
result.groupby("label").count().sort('count', ascending=False).toPandas()

In [ ]:
%%time

scc = cn.stronglyConnectedComponents(maxIter=10)

In [ ]:
scc.groupBy('component').count().sort('count', ascending=True).toPandas()